# Background

Inspired by my analysis of Miami Lakes Rboom data to measure the pressure pulse from the Tonga eruption an beamform back to the source, I began drinking in global barometric and infrasound data from IRIS (and Shake.net). Steve then took my measurements and began collating a table of reduced pressure versus other eruptions. This then mainfested an invite from Zhigang Peng to collaborate on a Tonga paper to look at all waves emanating from this eruption.

While I am focusing on that paper currently, this notebook has lots of earlier work, and will later be expanded for my own conference and journal papers (and one I am co-authoring with Steve).

Glenn Thompson, Jan - Feb 2022

# 1. Functions and imports

In [1]:
import os, sys, obspy
import numpy as np
%matplotlib inline
#import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pickle
LIBpath = os.path.join( os.getenv('HOME'),'src','kitchensinkGT', 'LIB')
sys.path.append(LIBpath)
from libseisGT import inventory2traceid, get_FDSN_inventory, clean_trace, \
    attach_station_coordinates_from_inventory
from obspy.clients.fdsn import Client
#from obspy.core.util import AttribDict
from obspy.geodetics import locations2degrees, degrees2kilometers, kilometers2degrees
from scipy.stats import linregress
from libTonga import * # if this does not work, try LIB/obsolete/Tonga_analayze_global_stations_seismicpaper.py        

# 2. Parameters

In [ ]:
DATA_ROOT = os.path.join(os.getenv('HOME'), 'DATA', 'Tonga_eruption_analysis')
if not os.path.isdir(DATA_ROOT):
    os.makedirs(DATA_ROOT)
searchRadiusDeg = 180
fmin = 0.0001
T = 1.0/fmin
circum_earth_km = 40075
circum_earth_m = circum_earth_km*1000
m_per_deg = circum_earth_m/360.0

# location of Hunga-Tonga Hunga-Ha'apai
olat = -(20 + 34/60 + 12 /3600) 
olon = -(175 + 22/60 + 48/3600)
otime = obspy.core.UTCDateTime('2022-01-15T04:14:45.000000Z') # main eruption time - on day 2

startt =  obspy.core.UTCDateTime('2022-01-13T00:00:00.000000Z') - T*2
endt = obspy.core.UTCDateTime('2022-01-20T00:00:00.000000Z') + T*2 # get 34 hours of data, since this is approximate time for sound waves to circle the earth

# 3. Run for barometers outside (LDO)

In [ ]:
pklfile = os.path.join(DATA_ROOT, 'st_ldo_downsample_pick.pkl')
if os.path.isfile(pklfile): # read data from pickle file and leap to 3.1.3
    with open(pklfile, 'rb') as f:
        st_ldo_downsample_pick = pickle.load(f)
else:
    fdsnURL = "http://service.iris.edu"
    chanstring = 'LDO'
    st_ldo, inv_ldo, raw_st_ldo, raw_inv_ldo = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

# to do:
- compute moveout speed of N wave peak times
- correct for actual circumference of wavefronts on a spherical earth

## 3.1 Pick N-wave onsets

In [ ]:
def process_1(st, otime=otime, cmin=300, cmax=430, decimation_factor=10):
    st_downsample = st.copy()
    st_downsample.decimate(decimation_factor)
    r = get_distance_vector(st_downsample)
    print(r)
    st_downsample_pick = order_traces_by_distance(st_downsample, r) 
    st_downsample_pick = manually_select_good_traces(st_downsample_pick, trim_mins=0, record_section=False, auto=False, wlen=7200)
    #pick_pressure_onset(st_downsample_pick, otime, cmin, cmax)
    return st_downsample_pick

### 3.1.1 Raw traces

In [ ]:
st_ldo_raw_downsample_pick = process_1(st_ldo_raw)

### 3.1.2 Reconstituted "good" traces

In [ ]:
#st_ldo_downsample_pick = process_1(st_ldo)

new_otime = obspy.UTCDateTime(2022,1,15,4,7,18)

for tr in st_ldo_downsample_pick:
    if tr.stats.network == 'PB':
        st_ldo_downsample_pick.remove(tr)

In [ ]:
plot_amplitude_versus_distance(st_ldo_downsample_pick, 'Pa', reftime=new_otime, \
                               cmin=300, cmax=420, duration=7200)
#plot_reduced_pressure(st, 'Pa')   
#plot_inv(inv)
#plots(st_ldo_downsample_pick, inv_ldo, olat, olon, otime, cmin=300, cmax=420)
print(len(st_ldo_downsample_pick))

### 3.1.3 Make record section of the N wave

In [ ]:
signal_duration = 3600.0 * 3.0
#wavespeed, new_otime, t, d = regress_arrival_times(st2, otime, outfile='Nwave_arrivalfit.eps')
wavespeed = 312.7
new_otime = obspy.UTCDateTime(2022,1,15,4,7,18)
etime = new_otime + (circum_earth_m/2)/wavespeed + signal_duration
#plot_record_section(st_ldo_downsample_pick, new_otime, starttime=new_otime, endtime=etime, \
#                    outfile='Nwave_recordsection.eps', slope_speed=wavespeed, figsize=(8,16), scale_factor=5)

In [ ]:
#plot_record_section(st_ldo_downsample_pick, new_otime, starttime=new_otime, endtime=etime, \
#                    outfile='Nwave_recordsection2.eps', slope_speed=wavespeed, figsize=(6,12), scale_factor=3)
reftime = obspy.UTCDateTime(2022,1,15,3,0,0)
stime = reftime
etime = obspy.UTCDateTime(2022,1,16,16,0,0)
celerity = 315.0
wraptime = circum_earth_m/celerity
plot_record_section(st_ldo_downsample_pick, reftime, starttime=stime, endtime=etime, \
                    outfile='Nwave_recordsection5.eps', slopes=[[celerity, otime], [-celerity, otime+wraptime]], figsize=(6,12), scale_factor=3, min_spacing_km=800)

In [ ]:
stime = obspy.UTCDateTime(2022,1,15,4,0,0)
etime = obspy.UTCDateTime(2022,1,15,6,0,0)
reftime = stime
plot_record_section(st_ldo_downsample_pick, reftime, starttime=stime, endtime=etime, \
    outfile='Nwave_recordsection3.eps', figsize=(12,12), scale_factor=1000, normalize=True, reduced_speed=315, min_spacing_km=1000)

### 3.1.4 make record section of multiple passes of N wave

In [ ]:
etime2 = obspy.UTCDateTime(2022,1,20,0,0,0)
st5 = st_ldo_downsample_pick.copy().filter('bandpass', freqmin=0.01, freqmax=0.1, corners=2)
plot_record_section(st5, new_otime, starttime=new_otime, endtime=etime2, \
    outfile='Nwave_recordsection4.eps', figsize=(16,16), scale_factor=10)

In [ ]:
stime2 = obspy.UTCDateTime(2022,1,15,0,0,0)
etime2 = obspy.UTCDateTime(2022,1,20,0,0,0)
st_ldo_lowpass = st_ldo_downsample_pick.copy().filter('bandpass', freqmin=0.0002, freqmax=0.01, corners=6)
wavespeed=310.0
wraptime = circum_earth_m/wavespeed 
plot_record_section(st_ldo_lowpass, stime2, starttime=stime2, endtime=etime2, \
    outfile='Nwave_recordsection_long_master.eps', \
                    slopes=[[wavespeed, new_otime],\
                            [-wavespeed, new_otime+wraptime], \
                            [wavespeed, new_otime+wraptime], \
                            [-wavespeed, new_otime+wraptime*2], \
                            [wavespeed, new_otime+wraptime*2], \
                            [-wavespeed, new_otime+wraptime*3], \
                            [wavespeed, new_otime+wraptime*3], \
                           ], \
                    figsize=(12,12), scale_factor=10, min_spacing_km = 500)

### 3.1.5 Save reconstituted, downsampled, picked barometric data to pickle file

In [ ]:
pklfile = os.path.join(DATA_ROOT, 'st_ldo_downsample_pick.pkl')
with open(pklfile, 'wb') as f:
    pickle.dump(st_ldo_downsample_pick, f)    

## 3.2 Explore the near stations only to improve origin time estimates

### 3.2.1 Read raw barometric data and order traces by distance

In [ ]:
pklfile = os.path.join(DATA_ROOT, 'st_ldo_raw.pkl')
if os.path.isfile(pklfile):
    with open(pklfile, 'rb') as f:
        st_ldo_raw_sorted = pickle.load(f)
else:
    #st_ldo_raw = obspy.core.read(os.path.join(DATA_ROOT, 'iris_LDO_within_180_degrees.mseed'))
    #inv_ldo_raw = obspy.read_inventory(os.path.join(DATA_ROOT, 'iris_LDO_within_180_degrees.xml'))
    #attach_station_coordinates_from_inventory(inv_ldo_raw, st_ldo_raw)
    #attach_distance_to_stream(st_ldo_raw, olat, olon)
    #r = get_distance_vector(st_ldo_raw)
    #st_ldo_raw_sorted = order_traces_by_distance(st_ldo_raw, r) 

    with open(pklfile, 'wb') as f:
        pickle.dump(st_ldo_raw_sorted, f) 

In [ ]:
r = get_distance_vector(st_ldo_raw_sorted)
st_sorted = order_traces_by_distance(st_ldo_raw_sorted, r) 
st2 = remove_duplicates(stkeep_sorted)
st2.detrend('linear')
st3 = st2.copy().trim(starttime=obspy.UTCDateTime(2022,1,15,3,0,0), endtime=obspy.UTCDateTime(2022,1,15,7,0,0))
#stkeep = manually_select_good_traces(st, trim_mins=0, record_section=False, auto=False, wlen=7200, rmax=3200000)    

In [ ]:


#stkeep2.plot(equal_scale=False)
stime = obspy.UTCDateTime(2022,1,15,4,0,0)
stkeep2.normalize()
plot_record_section(stkeep2, stime, \
    outfile='Nwave_raw_close.eps', figsize=(16,16), scale_factor=500, km_max=3500)

In [ ]:
stkeep3 = stkeep2.copy().filter('highpass', freq=0.01, corners=2)

plot_record_section(stkeep3, stime, \
    outfile='Nwave_raw_close.eps', figsize=(16,16), scale_factor=500, km_max=3500)

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
pick_pressure_onset(stkeep3, new_otime, 250, 400)

In [ ]:
regress_arrival_times(stkeep3, new_otime, outfile=None)

In [ ]:
plot_record_section(stkeep3, stime, \
    outfile='Nwave_raw_close.eps', figsize=(16,16), scale_factor=500, km_max=3500, slopes=[[295.7, obspy.UTCDateTime(2022,1,15,4,1,28)]])

In [ ]:
#stkeep2[1].plot(type='dayplot');
tr = st_ldo_raw_sorted[0].copy()
tr.trim(starttime=stime+1800, endtime=stime+7200)
tr.plot();

In [ ]:
757000/(42*60)

In [ ]:
# miscellaneous stuff

plots(st, inv_ldo, olat, olon, otime, minf=0, auto=True)

r = get_distance_vector(st_ldo)
st = order_traces_by_distance(st_ldo, r) 
# sort traces by distance
for tr in st:
    print(tr.id, tr.stats.distance)

plots(st, inv_ldo, olat, olon, otime, startt=otime-200000, endt=otime, auto=True, minf=0.001)



# 4. Run for seismic channels

In [ ]:
pklfile = os.path.join(DATA_ROOT, 'st_lhz_downsample_pick.pkl')
if os.path.isfile(pklfile):
    with open(pklfile, 'rb') as f:
        st_lhz_downsample_pick = pickle.load(f) 
    
else:

    fdsnURL = "http://service.iris.edu"
    chanstring = 'LHZ'
    st_lhz, inv_lhz, st_lhz_raw, inv_lhz_raw = analyze_clientchan(fdsnURL, chanstring, fmin=0.005, network='IU')

    st_lhz_downsample_pick = process_1(st_lhz)
    
    with open(pklfile, 'wb') as f:
        pickle.dump(st_lhz_downsample_pick, f)     

st_f = st_lhz_downsample_pick.copy().filter('bandpass', freqmin=0.001, freqmax=0.01, corners=2)
signal_duration = 3600.0 * 3.0
wavespeed = 312.7
new_otime = obspy.UTCDateTime(2022,1,15,4,7,18)
etime = new_otime + (circum_earth_m/2)/wavespeed + signal_duration
plot_record_section(st_f, new_otime, starttime=new_otime, endtime=etime, \
    outfile='seismicwave_recordsection.eps', figsize=(8,16), scale_factor=2e8, slope_speed=wavespeed)

etime = new_otime + 3 * 3600
plot_record_section(st_lhz_downsample_pick, new_otime, starttime=new_otime, endtime=etime, \
    outfile='seismicwave_recordsection_short.eps', figsize=(8,16), scale_factor=1e8)


  

In [ ]:
stime = obspy.UTCDateTime(2022,1,15,0,0,0)
etime = obspy.UTCDateTime(2022,1,17,0,0,0)
st_f2 = st_lhz_downsample_pick.copy().filter('bandpass', freqmin=0.005, freqmax=0.02, corners=2)
plot_record_section(st_f2, stime, starttime=stime, endtime=etime, \
    outfile='seismicwave_recordsection.eps', figsize=(6,12), scale_factor=6e8, min_spacing_km = 2000)

In [ ]:
st_lhz_raw = obspy.core.read(os.path.join(DATA_ROOT, 'iris_LHZ_within_180_degrees.mseed'))
inv_lhz_raw = obspy.read_inventory(os.path.join(DATA_ROOT, 'iris_LHZ_within_180_degrees.xml'))
attach_station_coordinates_from_inventory(inv_lhz_raw, st_lhz_raw)
attach_distance_to_stream(st_lhz_raw, olat, olon)
reftime = obspy.UTCDateTime(2022,1,15,4,0,0)
stime = reftime
etime = obspy.UTCDateTime(2022,1,15,6,0,0)
celerity = 300.0
wraptime = circum_earth_m/celerity

In [ ]:
st = st_lhz_raw.copy()
st.filter('bandpass', freqmin=0.005, freqmax=0.02)
st.trim(starttime=stime-3600,endtime=etime+3600)
st.normalize()

In [ ]:
st_f3=st_f2.copy().normalize()
plot_record_section(st_f3, reftime, starttime=stime, endtime=etime, \
    outfile='seismicwave_recordsection5.eps', slopes=[[celerity, otime], \
    [-celerity, otime+wraptime], [4000, otime]], figsize=(6,12), scale_factor=3000, min_spacing_km=500)

In [ ]:
st_f2 = st_lhz_downsample_pick.copy().filter('bandpass', freqmin=0.001, freqmax=0.02, corners=2)
st_f3 = st_f2.copy().trim(starttime=stime-3600, endtime=etime+3600)
st_f4 = st_f3.copy().normalize(global_max=False)


In [ ]:
reftime = obspy.UTCDateTime(2022,1,15,3,0,0)
stime = reftime
etime = obspy.UTCDateTime(2022,1,16,16,0,0)
celerity = 300.0
wraptime = circum_earth_m/celerity
plot_record_section(st_f4, reftime, starttime=stime, endtime=etime, \
    outfile='seismicwave_recordsection_1pass_master.eps', slopes=[[3600, otime, 'b-'],[315.0, otime, 'r-'], [celerity, otime, 'g-'], [-celerity, otime+wraptime, 'g-']], figsize=(6,12), scale_factor=1000, min_spacing_km=1000)

In [ ]:
r = get_distance_vector(st_lhz)
st_lhz_sorted = order_traces_by_distance(st_lhz, r) 
plots(st_lhz_sorted, inv_ldo, olat, olon, otime, startt=otime, endt=otime+12*3600, auto=True, minf=0.001, maxf=0.1)

r = get_distance_vector(st_ldo)
st_ldo_sorted = order_traces_by_distance(st_ldo, r) 
plots(st_ldo_sorted, inv_ldo, olat, olon, otime, startt=otime, endt=otime+12*3600, auto=True, minf=0.001, maxf=0.1)

r = get_distance_vector(st_ldo)
st_ldo_sorted = order_traces_by_distance(st_ldo, r) 
plots(st_ldo_sorted, inv_ldo, olat, olon, otime, startt=otime, endt=st_ldo[0].stats.endtime, auto=True, minf=0.00015, maxf=0.25)

# 4. Run for infrasound channels

In [ ]:
fdsnURL = "http://service.iris.edu"
chanstring = 'LDF'
st_ldf, inv_ldf, raw_st_ldf, raw_inv_ldf  = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)
plots(st_ldf, otime, otime+17*3600)

# 5. Run for Raspberry Booms

In [ ]:
fdsnURL = "http://fdsnws.raspberryshakedata.com"
chanstring = 'HDF'
st_boom, inv_boom, raw_st_boom, raw_inv_boom  = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

# 6. Combine results in single graph

In [ ]:
st_list = [st_ldf, st_ldo, st_boom]
inv_list = [inv_ldf, inv_ldo, inv_boom]
units = 'Pa'
plot_combined_versus_distance(st_list, inv_list, units, corrected=False)
plot_combined_versus_distance(st_list, inv_list, units, corrected=True)

In [ ]:
for st in st_list:
    print(len(st))

To do:
- downsample H?? and B?? channels to 1 sample per second
- apply data quality metrics to remove bad channels
- extra steps to remove spikes?
- detect events?
- examine as a function of low-cut frequency
- make maps with pygmt, e.g. with radius contours, or amplitude contours

In [ ]:
fmin=1/10000

fdsnURL = "http://service.iris.edu"
chanstring = 'LDO'
st_ldo2, inv_ldo2 = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

fdsnURL = "http://service.iris.edu"
chanstring = 'LDF'
st_ldf2, inv_ldf2 = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

fdsnURL = "http://fdsnws.raspberryshakedata.com"
chanstring = 'HDF'
st_boom2, inv_boom2 = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

In [ ]:
st_list = [st_ldf2, st_ldo2]
inv_list = [inv_ldf2, inv_ldo2]
units = 'Pa'
plot_combined_versus_distance(st_list, inv_list, units, corrected=False)
plot_combined_versus_distance(st_list, inv_list, units, corrected=True)

In [ ]:
# plot traces by distance - record section?
st = obspy.core.Stream()
for tr in st_ldo2:
    tr.stats['distance']=tr.stats.distance_in_km*1000.0
    tr.data=tr.data*10
    st.append(tr)
for tr in st_ldf2:
    tr.stats['distance']=tr.stats.distance_in_km*1000.0
    tr.data=tr.data*10
    st.append(tr)  
    

In [ ]:
sectionpng = os.path.join(DATA_ROOT,'section.png')
st.plot(type='section',title='',dpi=75,scale=15.0,reftime=otime,norm_method='stream',fillcolors=['r','b'],outfile=sectionpng)

In [ ]:
print(len(st_ldo2),len(st_ldf2),len(st))

# 7. Investigate filter bandwidth needed to see signal fully at MSVF

In [ ]:
st = obspy.core.read(os.path.join(DATA_ROOT, 'iris_LDO_within_180_degrees.mseed'))
#st.select(station='MSVF').plot()
inv = obspy.read_inventory(os.path.join(DATA_ROOT, 'iris_LDO_within_180_degrees.xml'))
msvf = st.select(station='MSVF')
msvf2=msvf.copy()
msvf2.remove_response(inventory=inv)
msvf2.plot()
tr = msvf2[0]

In [ ]:
f = np.power(1.2,np.arange(-55,-1))
f = f[f<0.5]
plt.figure()
print(f)
for ncorners in range(6):
    a = []
    for fmin in f:
        tr2 = tr.copy()
        if fmin>0.0:
            #print(fmin)
            tr2.filter('highpass',freq=fmin,corners=ncorners+1)
        a.append(np.max(np.abs(tr2.data)))
    labstr = "%d corners" % (ncorners+1)
    plt.semilogx(f,a,label=labstr)
plt.ylabel('Pa')
plt.xlabel('Low-pass corner (Hz)')
plt.legend()
plt.show()

In [ ]:
fmin = np.power(2.0,-14)
#f = np.power(2.0,np.arange(-13,-1))
f = np.power(1.2,np.arange(-55,-1))
f = f[f<=0.5]
f = f[f>fmin*2]
f = np.append(f,1.0)
print(f)
print(fmin)

plt.figure()
for ncorners in range(6):
    a = []
    for fmax in f:
        tr2 = tr.copy()
        if fmax<1.0:
            #print(fmin)
            tr2.filter('bandpass',freqmin=fmin, freqmax=fmax,corners=ncorners+1)
            
        else:
            tr2.filter('highpass',freq=fmin, corners=ncorners+1)
        a.append(np.max(np.abs(tr2.data)))
    labstr = "%d corners" % (ncorners+1)
    plt.semilogx(f,a,label=labstr)
plt.ylabel('Pa')
plt.xlabel('High-pass corner (Hz)')
plt.legend()
plt.show()

In [ ]:
tr2=msvf[0].copy()
pre_filt = [0.0001, 0.0002, 0.5, 0.1]
tr2.filter('bandpass', freqmin=0.0001, freqmax=0.5, corners=2)
tr2.remove_response(inventory=inv, plot=True, water_level=60)
#tr2.plot()

In [ ]:
tr2=msvf[0].copy()
tr2.plot()
pre_filt = [0.0001, 0.0002, 0.25, 0.5]
#tr2.filter('pass', freq=0.0001, corners=2)
tr2.remove_response(inventory=inv, pre_filt=pre_filt, plot=True, water_level=60)
tr2.plot();

# Miscellaneous

In [ ]:
mseedfile = os.path.join(DATA_ROOT, 'rboom_HDF_within_180_degrees.mseed')
inv = obspy.read_inventory(os.path.join(DATA_ROOT, 'rboom_HDF_within_180_degrees.xml'))
pre_filt = [0.0001, 0.0002, 0.25, 0.5]
N = 1000
count = 0
f = open(os.path.join(DATA_ROOT, 'rboom_good_HDF_ids_within_180_degrees.pkl'), "rb")
good_ids = pickle.load(f)
f.close()
good_st = obspy.core.Stream()
more_good = ['S89A5', 'R160F', 'RC93C', 'RA8D4', 'R9606', 'R40C1', 'R3A8A', 'R571C', 'R9CDF', 'RB83C', 'R0033']
for sta in more_good:
    good_ids.append('AM.%s.00.HDF' % sta)
import obspy
import io

reclen = 512
chunksize = 100000 * reclen # Around 50 MB

with io.open(mseedfile, "rb") as fh:
    while count<N:
        with io.BytesIO() as buf:
            c = fh.read(chunksize)
            if not c:
                break
            buf.write(c)
            buf.seek(0, 0)
            st = obspy.read(buf)
            count+=1
        # Do something useful!
        st.merge()
        for tr in st:
            if tr.id in good_ids:
                good_st.append(tr)
        #st.remove_response(inventory=inv, pre_filt=pre_filt, plot=True, water_level=60) 

In [ ]:
st_good = good_st.copy()
st_good.taper(0.01)
for tr in st_good:
    try:
        tr.filter('highpass', freq=0.0001, corners=2)
    except:
        pass
st_good = manually_select_good_traces(st_good) 

In [ ]:
#st.select(station='MSVF').plot()
pre_filt = [0.001, 0.002, 0.25, 0.5]
inv = obspy.read_inventory(os.path.join(DATA_ROOT, 'rboom_HDF_within_180_degrees.xml'))
st_good2 = st_good.copy()
for tr in st_good2:
    #tr.remove_response(inventory=inv, pre_filt=pre_filt, plot=True, water_level=60)
    tr.remove_response(inventory=inv, pre_filt=pre_filt, plot=True, water_level=60)
    tr.plot();

In [ ]:
tr = good_st.copy()[0]
tr.plot();

In [ ]:
tr.trim(starttime=obspy.UTCDateTime(2022,1,15,18,0,0), endtime=obspy.UTCDateTime(2022,1,15,23,0,0))
tr.plot()

In [ ]:
tr.remove_response(inventory=inv, pre_filt=[0.01, 0.02, 10, 20], plot=True, water_level=60)

In [ ]:
inv_ldo3 = inv_ldo2.copy()
inv_ldo3.__add__(inv_ldf2)

In [ ]:
plot_inv(inv_ldo3)

In [ ]:
msvf

In [ ]:
st_ldo.select(station='MSVF')[0].stats.distance_in_km

In [ ]:
plot_inv(inv_ldo3)

In [ ]:
searchRadiusDeg = 20
fmin = 0.0001
T = 1.0/fmin

# location of Hunga-Tonga Hunga-Ha'apai
olat = -(20 + 34/60 + 12 /3600) 
olon = -(175 + 22/60 + 48/3600)
otime = obspy.core.UTCDateTime('2022-01-15T04:14:45.000000Z') # main eruption time - on day 2

startt = otime - T*2
endt = otime + 7 * 3600 + T*2 

fdsnURL = "http://service.iris.edu"
chanstring = 'HHZ'
st_hhz, inv_hhz = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

In [ ]:
st=st_hhz.copy()
st.taper(0.1)
st.filter('bandpass', freqmin=0.01, freqmax=1.0, corners=2)
st.trim(starttime=obspy.UTCDateTime(2022,1,15,4,14,45),endtime=obspy.UTCDateTime(2022,1,15,7,14,45))
st.plot(equal_scale=False);

In [ ]:
for tr in st_hhz:
    print(tr.stats.distance_in_km)

In [ ]:
st.trim(starttime=obspy.UTCDateTime(2022,1,15,6,53,0),endtime=obspy.UTCDateTime(2022,1,15,6,57,0))
st.plot(equal_scale=False);

In [ ]:
chanstring = 'BHZ'
st_bhz, inv_bhz = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

In [ ]:
st=st_bhz.copy()
st.taper(0.1)
st.filter('bandpass', freqmin=0.01, freqmax=15.0, corners=2)
st.trim(starttime=obspy.UTCDateTime(2022,1,15,4,14,45),endtime=obspy.UTCDateTime(2022,1,15,4,24,45))
st.plot(equal_scale=False);

In [ ]:
searchRadiusDeg = 60
fmin = 0.001
T = 1.0/fmin

# location of Hunga-Tonga Hunga-Ha'apai
olat = -(20 + 34/60 + 12 /3600) 
olon = -(175 + 22/60 + 48/3600)
otime = obspy.core.UTCDateTime('2022-01-15T04:14:45.000000Z') # main eruption time - on day 2

distkm = degrees2kilometers(searchRadiusDeg)
max_acoustic_speed = 380
min_acoustic_speed = 300
min_travel_time = distkm * 1000 / max_acoustic_speed 
max_travel_time = distkm * 1000 / min_acoustic_speed                   
print(searchRadiusDeg, distkm, max_travel_time, min_travel_time)

startt = otime - T*2
endt = otime + max_travel_time + T*2 

fdsnURL = "http://service.iris.edu"
chanstring = 'LHZ'
st_lhz, inv_lhz = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

In [ ]:
st_lhz

In [ ]:
st = st_lhz.copy()
st.taper(0.1)
st.filter('bandpass', freqmin=0.003, freqmax=0.5, corners=2)
st.trim(starttime=obspy.UTCDateTime(2022,1,15,4,0,0),endtime=obspy.UTCDateTime(2022,1,15,5,0,0))
for tr in st:
    tr.plot()


In [ ]:
st.select(station='MSVF').plot();

In [ ]:
tr = obspy.core.Trace()
tr.data = st[0].data + st[1].data
tr.stats.starttime = st[0].stats.starttime
tr.stats.sampling_rate = st[0].stats.sampling_rate
tr.id = 'XX.MSVF.20.LHZ'
tr.plot()

In [ ]:
tr.trim(starttime=obspy.UTCDateTime(2022,1,15,4,15,0),endtime=obspy.UTCDateTime(2022,1,15,4,35,0))
tr.plot()

In [ ]:
fdsnClient = Client(fdsnURL)  
startt = otime-10000
endt = obspy.UTCDateTime(2022,1,15,7,0,0)+10000

msvf = fdsnClient.get_waveforms('II', 'MSVF', '*', '*', startt, endt)

In [ ]:
for loc in ['00', '10']:
    st = msvf.select(location=loc, channel='[LV][HD]?')
    st.plot(equal_scale=False);

In [ ]:
for tr in msvf:
    print(tr.id, tr.stats.sampling_rate)

In [ ]:
for loc in ['00', '10']:
    st = msvf.copy().select(location=loc, channel='[LV][HD]?')
    st.detrend('linear')
    st.taper(0.02)
    st.filter('bandpass',freqmin=0.0002,freqmax=0.01)
    st.plot(equal_scale=False);

In [ ]:
print(msvf)

In [ ]:
for loc in ['00', '10']:
    st = msvf.copy().select(location=loc, channel='B[HD]?')
    st.detrend('linear')
    st.taper(0.02)
    st.filter('highpass',freq=0.5,corners=2)
    st.trim(starttime=otime+150,endtime=otime+300)
    st.plot(equal_scale=False);

In [ ]:
searchRadiusDeg = 120
fmin = 0.001
T = 1.0/fmin

# location of Hunga-Tonga Hunga-Ha'apai
olat = -(20 + 34/60 + 12 /3600) 
olon = -(175 + 22/60 + 48/3600)
otime = obspy.core.UTCDateTime('2022-01-15T04:14:45.000000Z') # main eruption time - on day 2

distkm = degrees2kilometers(searchRadiusDeg)
max_acoustic_speed = 380
min_acoustic_speed = 300
min_travel_time = distkm * 1000 / max_acoustic_speed 
max_travel_time = distkm * 1000 / min_acoustic_speed                   
print(searchRadiusDeg, distkm, max_travel_time, min_travel_time)

startt = otime - T*2
endt = otime + max_travel_time + T*2 

fdsnURL = "http://service.iris.edu"
chanstring = 'LHZ'
st_lhz, inv_lhz = analyze_clientchan(fdsnURL, chanstring, fmin=fmin)

In [ ]:
print(DATA_ROOT)

In [ ]:
st_ldf_recon = obspy.read(os.path.join(DATA_ROOT, 'iris_LDF_within_180_degrees.mseed'))

In [ ]:
len(st_ldf_recon)

In [ ]:
len(st_ldf_recon.merge())

In [ ]:
ids = []
for tr in st_ldf_recon:
    if not tr.id in ids:
        ids.append(tr.id)
print(len(ids))

In [ ]:
print(ids)

In [ ]:
st_ldo_recon = obspy.read(os.path.join(DATA_ROOT, 'iris_LDO_within_180_degrees.mseed'))

In [ ]:
ids = []
for tr in st_ldo_recon:
    if not tr.id in ids:
        ids.append(tr.id)
print(len(ids))

In [ ]:
757000/340

In [ ]:
2226/60